# 02 - Constructors in Python (__init__)

> **Phase 3: Object-Oriented Programming & Modules (Tháng 3)**

---

## 📚 Mục tiêu học tập

Sau bài học này, bạn sẽ:

1. Hiểu rõ constructor là gì và vai trò trong OOP
2. Nắm vững cách sử dụng `__init__` method
3. Hiểu quy trình tạo object: `__new__` vs `__init__`
4. Làm việc với default parameters và *args, **kwargs
5. Tạo multiple constructors với @classmethod
6. Áp dụng validation patterns trong constructor
7. Sử dụng dataclasses như alternative cho boilerplate code

---

## 1. Constructor là gì?

### 1.1 Định nghĩa

**Constructor** là một method đặc biệt được gọi **tự động** khi tạo object mới từ class.

Trong Python:
- **`__init__`** là **initializer** - khởi tạo giá trị cho object đã được tạo
- **`__new__`** là **constructor thực sự** - tạo và trả về object mới

> 💡 Trong thực tế, chúng ta thường chỉ làm việc với `__init__` và gọi nó là "constructor".

### 1.2 Mục đích của Constructor

| Mục đích | Mô tả |
|----------|-------|
| **Khởi tạo attributes** | Đặt giá trị ban đầu cho các thuộc tính |
| **Thiết lập trạng thái** | Cấu hình object ở trạng thái hợp lệ |
| **Validate dữ liệu** | Kiểm tra input trước khi lưu |
| **Acquire resources** | Mở file, kết nối database... |
| **Dependency injection** | Nhận các dependencies từ bên ngoài |

### 1.3 So sánh: Có vs Không có Constructor

In [ ]:
# ❌ KHÔNG có constructor - phải set attributes thủ công
class PersonWithoutInit:
    pass

p1 = PersonWithoutInit()
p1.name = "John"  # Phải set sau khi tạo
p1.age = 25

p2 = PersonWithoutInit()
# Quên set attributes -> AttributeError khi truy cập
try:
    print(p2.name)
except AttributeError as e:
    print(f"Error: {e}")

print(f"p1: {p1.name}, {p1.age}")

In [ ]:
# ✅ CÓ constructor - khởi tạo ngay khi tạo object
class PersonWithInit:
    def __init__(self, name, age):
        self.name = name
        self.age = age

# Bắt buộc phải cung cấp name và age
p1 = PersonWithInit("Jane", 30)
print(f"p1: {p1.name}, {p1.age}")

# Không thể tạo object thiếu arguments
try:
    p2 = PersonWithInit()  # TypeError!
except TypeError as e:
    print(f"Error: {e}")

---

## 2. Cú pháp và Quy tắc của __init__

### 2.1 Cú pháp cơ bản

```python
class ClassName:
    def __init__(self, param1, param2, ...):
        self.attribute1 = param1
        self.attribute2 = param2
        # Có thể khởi tạo thêm attributes
        self.other_attr = some_default_value
```

### 2.2 Quy tắc quan trọng

In [ ]:
class DemoRules:
    """Demo các quy tắc của __init__."""
    
    def __init__(self, value):
        # Quy tắc 1: self luôn là parameter đầu tiên
        # Quy tắc 2: Không return giá trị (chỉ None được phép)
        # Quy tắc 3: Được gọi TỰ ĐỘNG sau khi object được tạo
        
        print(f"__init__ called with value={value}")
        self.value = value
        
        # ❌ SAI: return giá trị khác None
        # return self  # TypeError!
        
        # ✅ ĐÚNG: có thể return None (hoặc không return gì)
        return None

obj = DemoRules(42)
print(f"Object value: {obj.value}")

### 2.3 __init__ được gọi khi nào?

In [ ]:
class TrackedCreation:
    creation_count = 0
    
    def __init__(self, name):
        TrackedCreation.creation_count += 1
        self.name = name
        self.id = TrackedCreation.creation_count
        print(f"Created object #{self.id}: {self.name}")

# __init__ được gọi mỗi khi tạo object
a = TrackedCreation("Alpha")
b = TrackedCreation("Beta")
c = TrackedCreation("Gamma")

print(f"\nTotal objects created: {TrackedCreation.creation_count}")

---

## 3. Parameters trong Constructor

### 3.1 Required Parameters (Bắt buộc)

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        """Cả width và height đều bắt buộc."""
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height

# Phải cung cấp cả 2 arguments
rect = Rectangle(5, 3)
print(f"Rectangle {rect.width}x{rect.height}, Area: {rect.area()}")

# Thiếu argument -> Error
try:
    rect2 = Rectangle(5)  # Missing height
except TypeError as e:
    print(f"Error: {e}")

### 3.2 Default Parameters (Giá trị mặc định)

In [ ]:
class Student:
    def __init__(self, name, age=18, major="Undeclared", gpa=0.0):
        """
        Args:
            name: Tên sinh viên (bắt buộc)
            age: Tuổi (mặc định 18)
            major: Ngành học (mặc định 'Undeclared')
            gpa: Điểm trung bình (mặc định 0.0)
        """
        self.name = name
        self.age = age
        self.major = major
        self.gpa = gpa
    
    def __str__(self):
        return f"{self.name}, {self.age}yo, {self.major}, GPA: {self.gpa}"

# Các cách tạo Student
s1 = Student("Alice")                           # Chỉ name
s2 = Student("Bob", 20)                         # name + age
s3 = Student("Charlie", 22, "Computer Science") # name + age + major
s4 = Student("Diana", major="Mathematics")      # Skip age, dùng keyword
s5 = Student("Eve", gpa=3.8, age=21)            # Mix positional và keyword

for student in [s1, s2, s3, s4, s5]:
    print(student)

### 3.3 ⚠️ Mutable Default Argument Trap

**QUAN TRỌNG**: Không dùng mutable objects (list, dict, set) làm default value!

In [ ]:
# ❌ SAI: Mutable default argument
class BadClass:
    def __init__(self, items=[]):  # List được shared giữa tất cả instances!
        self.items = items

b1 = BadClass()
b1.items.append("item1")

b2 = BadClass()  # Không truyền items
print(f"b2.items: {b2.items}")  # ['item1'] - Sai! Mong đợi []

b1.items.append("item2")
print(f"b1.items: {b1.items}")
print(f"b2.items: {b2.items}")  # Cũng thay đổi theo!

In [ ]:
# ✅ ĐÚNG: Dùng None làm default, tạo mới trong __init__
class GoodClass:
    def __init__(self, items=None):
        self.items = items if items is not None else []
        # Hoặc: self.items = items or []  # Nhưng cẩn thận với empty list
        # Hoặc: self.items = [] if items is None else items

g1 = GoodClass()
g1.items.append("item1")

g2 = GoodClass()
print(f"g1.items: {g1.items}")
print(f"g2.items: {g2.items}")  # [] - Đúng!

### 3.4 *args và **kwargs trong Constructor

In [ ]:
class FlexibleConfig:
    """Class với constructor linh hoạt."""
    
    def __init__(self, name, *args, **kwargs):
        """
        Args:
            name: Tên config (bắt buộc)
            *args: Các giá trị bổ sung
            **kwargs: Các cặp key-value bổ sung
        """
        self.name = name
        self.extra_values = args
        
        # Gán tất cả kwargs thành attributes
        for key, value in kwargs.items():
            setattr(self, key, value)
    
    def show_all(self):
        print(f"Name: {self.name}")
        print(f"Extra values: {self.extra_values}")
        print(f"All attributes: {vars(self)}")

# Sử dụng linh hoạt
config1 = FlexibleConfig("App Config", 1, 2, 3, debug=True, version="1.0")
config1.show_all()

print(f"\nconfig1.debug: {config1.debug}")
print(f"config1.version: {config1.version}")

In [ ]:
# Ví dụ thực tế: HTML Element
class HTMLElement:
    def __init__(self, tag, content="", **attributes):
        self.tag = tag
        self.content = content
        self.attributes = attributes
    
    def render(self):
        attrs = " ".join(f'{k}="{v}"' for k, v in self.attributes.items())
        attrs_str = f" {attrs}" if attrs else ""
        return f"<{self.tag}{attrs_str}>{self.content}</{self.tag}>"

# Tạo các HTML elements
div = HTMLElement("div", "Hello World", id="main", class_="container")
link = HTMLElement("a", "Click me", href="https://example.com", target="_blank")
img = HTMLElement("img", src="photo.jpg", alt="A photo", width="100")

print(div.render())
print(link.render())
print(img.render())

### 3.5 Keyword-Only Arguments

In [ ]:
class Connection:
    def __init__(self, host, port, *, timeout=30, retry=3, ssl=False):
        """
        Args:
            host: Server host (positional)
            port: Server port (positional)
            timeout: Connection timeout (keyword-only)
            retry: Number of retries (keyword-only)
            ssl: Use SSL (keyword-only)
        
        Note: * forces everything after it to be keyword-only
        """
        self.host = host
        self.port = port
        self.timeout = timeout
        self.retry = retry
        self.ssl = ssl
    
    def __str__(self):
        protocol = "https" if self.ssl else "http"
        return f"{protocol}://{self.host}:{self.port} (timeout={self.timeout}s, retry={self.retry})"

# Positional args cho host, port; keyword args cho options
conn1 = Connection("localhost", 8080)
conn2 = Connection("api.example.com", 443, ssl=True, timeout=60)

print(conn1)
print(conn2)

# ❌ Không thể truyền options như positional
try:
    conn3 = Connection("host", 80, 10)  # Error!
except TypeError as e:
    print(f"\nError: {e}")

---

## 4. Validation trong Constructor

### 4.1 Basic Validation

In [ ]:
class Person:
    def __init__(self, name, age, email):
        # Validate name
        if not name or not isinstance(name, str):
            raise ValueError("Name must be a non-empty string")
        
        # Validate age
        if not isinstance(age, int) or age < 0 or age > 150:
            raise ValueError("Age must be an integer between 0 and 150")
        
        # Validate email
        if not email or "@" not in email or "." not in email:
            raise ValueError("Invalid email format")
        
        self.name = name.strip().title()
        self.age = age
        self.email = email.lower().strip()
    
    def __str__(self):
        return f"{self.name}, {self.age}, {self.email}"

# Valid
p1 = Person("john doe  ", 30, "John.Doe@Email.COM")
print(f"Valid: {p1}")

# Invalid examples
test_cases = [
    ("", 25, "test@email.com"),      # Empty name
    ("Alice", -5, "alice@email.com"), # Negative age
    ("Bob", 30, "invalid-email"),     # Bad email
]

for name, age, email in test_cases:
    try:
        Person(name, age, email)
    except ValueError as e:
        print(f"Invalid ({name}, {age}, {email}): {e}")

### 4.2 Validation với Custom Exceptions

In [ ]:
# Custom exceptions
class ValidationError(Exception):
    """Base validation error."""
    pass

class InvalidAgeError(ValidationError):
    """Age validation error."""
    pass

class InvalidEmailError(ValidationError):
    """Email validation error."""
    pass

class User:
    def __init__(self, username, age, email):
        self.username = self._validate_username(username)
        self.age = self._validate_age(age)
        self.email = self._validate_email(email)
    
    @staticmethod
    def _validate_username(username):
        if not username or len(username) < 3:
            raise ValidationError("Username must be at least 3 characters")
        if not username.isalnum():
            raise ValidationError("Username must be alphanumeric")
        return username.lower()
    
    @staticmethod
    def _validate_age(age):
        if not isinstance(age, int):
            raise InvalidAgeError("Age must be an integer")
        if age < 13:
            raise InvalidAgeError("Must be at least 13 years old")
        if age > 120:
            raise InvalidAgeError("Invalid age")
        return age
    
    @staticmethod
    def _validate_email(email):
        import re
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(pattern, email):
            raise InvalidEmailError(f"Invalid email: {email}")
        return email.lower()
    
    def __repr__(self):
        return f"User('{self.username}', {self.age}, '{self.email}')"

# Test
try:
    user = User("JohnDoe123", 25, "john@example.com")
    print(f"Valid: {user}")
except ValidationError as e:
    print(f"Error: {e}")

# Test invalid age
try:
    User("alice", 10, "alice@test.com")
except InvalidAgeError as e:
    print(f"Age error: {e}")

### 4.3 Validation với Property Setters

In [ ]:
class BankAccount:
    """Bank account với validation qua property setters."""
    
    def __init__(self, owner, balance=0):
        # Property setters sẽ validate
        self.owner = owner
        self.balance = balance
        self._transactions = []
    
    @property
    def owner(self):
        return self._owner
    
    @owner.setter
    def owner(self, value):
        if not value or not isinstance(value, str):
            raise ValueError("Owner must be a non-empty string")
        self._owner = value.strip().title()
    
    @property
    def balance(self):
        return self._balance
    
    @balance.setter
    def balance(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("Balance must be a number")
        if value < 0:
            raise ValueError("Balance cannot be negative")
        self._balance = float(value)
    
    def __str__(self):
        return f"Account: {self.owner} - ${self.balance:,.2f}"

# Valid
acc = BankAccount("john doe", 1000)
print(acc)

# Validation works even after creation!
try:
    acc.balance = -500  # Error!
except ValueError as e:
    print(f"Error: {e}")

---

## 5. Alternative Constructors với @classmethod

Python không hỗ trợ **constructor overloading** (nhiều `__init__` với signatures khác nhau), nhưng có thể dùng `@classmethod` để tạo **alternative constructors**.

### 5.1 Pattern cơ bản

In [ ]:
from datetime import date, datetime

class Person:
    """Person class với multiple constructors."""
    
    def __init__(self, name, age):
        """Primary constructor."""
        self.name = name
        self.age = age
    
    @classmethod
    def from_birth_year(cls, name, birth_year):
        """Tạo Person từ năm sinh."""
        age = date.today().year - birth_year
        return cls(name, age)  # Gọi __init__
    
    @classmethod
    def from_birth_date(cls, name, birth_date):
        """Tạo Person từ ngày sinh (date object)."""
        today = date.today()
        age = today.year - birth_date.year
        # Điều chỉnh nếu chưa đến sinh nhật
        if (today.month, today.day) < (birth_date.month, birth_date.day):
            age -= 1
        return cls(name, age)
    
    @classmethod
    def from_dict(cls, data):
        """Tạo Person từ dictionary."""
        return cls(data['name'], data['age'])
    
    @classmethod
    def from_string(cls, person_str):
        """Tạo Person từ string 'name:age'."""
        parts = person_str.split(':')
        if len(parts) != 2:
            raise ValueError("String format should be 'name:age'")
        return cls(parts[0].strip(), int(parts[1]))
    
    def __str__(self):
        return f"{self.name}, {self.age} years old"

# Các cách tạo Person
p1 = Person("Alice", 25)                                    # Standard
p2 = Person.from_birth_year("Bob", 1995)                    # Từ năm sinh
p3 = Person.from_birth_date("Charlie", date(1990, 6, 15))   # Từ ngày sinh
p4 = Person.from_dict({"name": "Diana", "age": 30})         # Từ dict
p5 = Person.from_string("Eve : 28")                         # Từ string

for p in [p1, p2, p3, p4, p5]:
    print(p)

### 5.2 Ví dụ: Date Class với nhiều constructors

In [ ]:
from datetime import datetime, date
import calendar

class MyDate:
    """Custom Date class với nhiều ways to construct."""
    
    WEEKDAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    def __init__(self, year, month, day):
        # Validation
        if not (1 <= month <= 12):
            raise ValueError(f"Month must be 1-12, got {month}")
        
        max_day = calendar.monthrange(year, month)[1]
        if not (1 <= day <= max_day):
            raise ValueError(f"Day must be 1-{max_day} for {year}/{month}")
        
        self.year = year
        self.month = month
        self.day = day
    
    @classmethod
    def today(cls):
        """Current date."""
        d = date.today()
        return cls(d.year, d.month, d.day)
    
    @classmethod
    def from_string(cls, date_str, fmt="%Y-%m-%d"):
        """Parse from string."""
        dt = datetime.strptime(date_str, fmt)
        return cls(dt.year, dt.month, dt.day)
    
    @classmethod
    def from_timestamp(cls, timestamp):
        """From Unix timestamp."""
        dt = datetime.fromtimestamp(timestamp)
        return cls(dt.year, dt.month, dt.day)
    
    @classmethod
    def from_iso_format(cls, iso_string):
        """From ISO format string (YYYY-MM-DD)."""
        return cls.from_string(iso_string, "%Y-%m-%d")
    
    @classmethod
    def first_of_month(cls, year, month):
        """First day of given month."""
        return cls(year, month, 1)
    
    @classmethod
    def last_of_month(cls, year, month):
        """Last day of given month."""
        last_day = calendar.monthrange(year, month)[1]
        return cls(year, month, last_day)
    
    @property
    def weekday(self):
        """Day of week."""
        d = date(self.year, self.month, self.day)
        return self.WEEKDAYS[d.weekday()]
    
    @property
    def is_weekend(self):
        return self.weekday in ['Saturday', 'Sunday']
    
    def __str__(self):
        return f"{self.year:04d}-{self.month:02d}-{self.day:02d}"
    
    def __repr__(self):
        return f"MyDate({self.year}, {self.month}, {self.day})"

# Demo
print("=== MyDate Demo ===")
d1 = MyDate(2026, 1, 25)
d2 = MyDate.today()
d3 = MyDate.from_string("25/12/2025", "%d/%m/%Y")
d4 = MyDate.first_of_month(2026, 2)
d5 = MyDate.last_of_month(2026, 2)

print(f"d1: {d1} ({d1.weekday})")
print(f"d2 (today): {d2}")
print(f"d3: {d3}")
print(f"d4 (first of Feb): {d4}")
print(f"d5 (last of Feb): {d5}")
print(f"d1 is weekend: {d1.is_weekend}")

### 5.3 Ví dụ: Money Class

In [ ]:
class Money:
    """Represents monetary values with currency."""
    
    # Exchange rates to USD
    EXCHANGE_RATES = {
        'USD': 1.0,
        'EUR': 1.08,
        'GBP': 1.26,
        'VND': 0.00004,
        'JPY': 0.0067
    }
    
    def __init__(self, amount, currency='USD'):
        if currency not in self.EXCHANGE_RATES:
            raise ValueError(f"Unknown currency: {currency}")
        self.amount = float(amount)
        self.currency = currency
    
    @classmethod
    def dollars(cls, amount):
        return cls(amount, 'USD')
    
    @classmethod
    def euros(cls, amount):
        return cls(amount, 'EUR')
    
    @classmethod
    def vnd(cls, amount):
        return cls(amount, 'VND')
    
    @classmethod
    def from_string(cls, money_str):
        """Parse '100 USD' or 'USD 100'."""
        parts = money_str.strip().split()
        if len(parts) != 2:
            raise ValueError("Format: 'amount currency' or 'currency amount'")
        
        try:
            amount = float(parts[0])
            currency = parts[1].upper()
        except ValueError:
            amount = float(parts[1])
            currency = parts[0].upper()
        
        return cls(amount, currency)
    
    def to_usd(self):
        """Convert to USD."""
        rate = self.EXCHANGE_RATES[self.currency]
        return Money(self.amount * rate, 'USD')
    
    def convert_to(self, target_currency):
        """Convert to another currency."""
        usd_amount = self.to_usd().amount
        target_rate = self.EXCHANGE_RATES[target_currency]
        return Money(usd_amount / target_rate, target_currency)
    
    def __str__(self):
        return f"{self.amount:,.2f} {self.currency}"
    
    def __repr__(self):
        return f"Money({self.amount}, '{self.currency}')"

# Demo
m1 = Money(100, 'USD')
m2 = Money.euros(85)
m3 = Money.vnd(2_500_000)
m4 = Money.from_string("50 GBP")

print(f"m1: {m1}")
print(f"m2: {m2} = {m2.to_usd()}")
print(f"m3: {m3} = {m3.to_usd()}")
print(f"m4: {m4} = {m4.convert_to('EUR')}")

---

## 6. __new__ vs __init__ - Object Creation Lifecycle

### 6.1 Quy trình tạo Object trong Python

```
MyClass(args)  →  __new__(cls, args)  →  __init__(self, args)  →  object
                       ↓                        ↓
                 Tạo instance              Khởi tạo values
                 (allocate memory)         (initialize state)
```

In [ ]:
class LifecycleDemo:
    """Demo quy trình tạo object."""
    
    def __new__(cls, value):
        print(f"1. __new__ called with cls={cls.__name__}, value={value}")
        # Tạo instance mới
        instance = super().__new__(cls)
        print(f"   Instance created: {instance}")
        return instance
    
    def __init__(self, value):
        print(f"2. __init__ called with self={self}, value={value}")
        self.value = value
        print(f"   self.value set to {self.value}")

print("Creating object...")
obj = LifecycleDemo(42)
print(f"\n3. Object created: {obj}, value={obj.value}")

### 6.2 Khi nào dùng __new__?

- **Singleton pattern**: Chỉ cho phép 1 instance
- **Immutable objects**: Subclassing int, str, tuple...
- **Factory pattern**: Trả về subclass dựa trên input
- **Object caching/pooling**: Reuse existing objects

In [ ]:
# Singleton Pattern
class Singleton:
    """Chỉ tạo 1 instance duy nhất."""
    
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            print("Creating the ONE and ONLY instance")
            cls._instance = super().__new__(cls)
        else:
            print("Returning existing instance")
        return cls._instance
    
    def __init__(self, value=None):
        # __init__ vẫn được gọi mỗi lần!
        if value is not None:
            self.value = value

s1 = Singleton("first")
s2 = Singleton("second")
s3 = Singleton()

print(f"\ns1 is s2: {s1 is s2}")
print(f"s1.value: {s1.value}")
print(f"s2.value: {s2.value}")  # Bị override bởi lần gọi thứ 2!

In [ ]:
# Better Singleton - chỉ init một lần
class BetterSingleton:
    _instance = None
    _initialized = False
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self, value=None):
        # Chỉ init một lần
        if not BetterSingleton._initialized:
            self.value = value
            BetterSingleton._initialized = True
            print(f"Initialized with value={value}")
        else:
            print("Already initialized, ignoring")

b1 = BetterSingleton("first")
b2 = BetterSingleton("second")  # Ignored
print(f"\nb1.value: {b1.value}")
print(f"b2.value: {b2.value}")

In [ ]:
# Factory Pattern với __new__
class Shape:
    """Factory tạo shape dựa trên input."""
    
    def __new__(cls, shape_type, *args):
        if cls is Shape:  # Chỉ khi gọi Shape(), không phải subclass
            if shape_type == 'circle':
                return super().__new__(Circle)
            elif shape_type == 'rectangle':
                return super().__new__(Rectangle)
            elif shape_type == 'square':
                return super().__new__(Square)
            else:
                raise ValueError(f"Unknown shape: {shape_type}")
        return super().__new__(cls)

class Circle(Shape):
    def __init__(self, _, radius):
        self.radius = radius
    def area(self):
        import math
        return math.pi * self.radius ** 2

class Rectangle(Shape):
    def __init__(self, _, width, height):
        self.width = width
        self.height = height
    def area(self):
        return self.width * self.height

class Square(Shape):
    def __init__(self, _, side):
        self.side = side
    def area(self):
        return self.side ** 2

# Factory in action
s1 = Shape('circle', 5)
s2 = Shape('rectangle', 4, 6)
s3 = Shape('square', 3)

print(f"s1: {type(s1).__name__}, area={s1.area():.2f}")
print(f"s2: {type(s2).__name__}, area={s2.area():.2f}")
print(f"s3: {type(s3).__name__}, area={s3.area():.2f}")

---

## 7. Dataclasses - Modern Alternative

Python 3.7+ cung cấp `@dataclass` decorator để tự động generate `__init__`, `__repr__`, `__eq__`...

In [ ]:
from dataclasses import dataclass, field
from typing import List, Optional

# Traditional class
class TraditionalPerson:
    def __init__(self, name, age, email=None):
        self.name = name
        self.age = age
        self.email = email
    
    def __repr__(self):
        return f"TraditionalPerson(name='{self.name}', age={self.age}, email='{self.email}')"
    
    def __eq__(self, other):
        if isinstance(other, TraditionalPerson):
            return self.name == other.name and self.age == other.age
        return False

# Dataclass - much cleaner!
@dataclass
class ModernPerson:
    name: str
    age: int
    email: Optional[str] = None

# Comparison
tp = TraditionalPerson("Alice", 25, "alice@email.com")
mp = ModernPerson("Alice", 25, "alice@email.com")

print(f"Traditional: {tp}")
print(f"Modern: {mp}")

# Equality works automatically
mp2 = ModernPerson("Alice", 25, "different@email.com")
print(f"\nmp == mp2: {mp == mp2}")  # True - chỉ so name, age, email

In [ ]:
# Advanced dataclass features
from dataclasses import dataclass, field
from typing import List
from datetime import datetime

@dataclass
class Product:
    name: str
    price: float
    quantity: int = 0
    tags: List[str] = field(default_factory=list)  # Mutable default
    created_at: datetime = field(default_factory=datetime.now)
    _id: str = field(default="", repr=False)  # Không hiện trong repr
    
    def __post_init__(self):
        """Called after __init__, useful for validation/processing."""
        if self.price < 0:
            raise ValueError("Price cannot be negative")
        if not self._id:
            import uuid
            self._id = str(uuid.uuid4())[:8]
    
    @property
    def total_value(self):
        return self.price * self.quantity

p1 = Product("Laptop", 999.99, 5, ["electronics", "computer"])
p2 = Product("Mouse", 29.99)

print(p1)
print(p2)
print(f"\np1._id: {p1._id}")
print(f"p1.total_value: ${p1.total_value}")

In [ ]:
# Frozen dataclass (immutable)
@dataclass(frozen=True)
class Point:
    x: float
    y: float
    
    @property
    def distance_from_origin(self):
        return (self.x ** 2 + self.y ** 2) ** 0.5

p = Point(3, 4)
print(f"Point: {p}")
print(f"Distance: {p.distance_from_origin}")

# Cannot modify frozen dataclass
try:
    p.x = 10  # Error!
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")

# Can use as dict key (hashable)
points = {p: "origin nearby"}
print(f"Dict with frozen point: {points}")

---

## 8. Ví dụ thực tế toàn diện

### 8.1 Configuration Class

In [ ]:
import json
import os
from dataclasses import dataclass, field, asdict
from typing import Dict, Any, Optional

@dataclass
class DatabaseConfig:
    host: str = "localhost"
    port: int = 5432
    database: str = "mydb"
    username: str = "user"
    password: str = field(default="", repr=False)  # Hide password in repr
    
    @property
    def connection_string(self):
        return f"postgresql://{self.username}:{self.password}@{self.host}:{self.port}/{self.database}"

@dataclass
class AppConfig:
    app_name: str
    debug: bool = False
    log_level: str = "INFO"
    database: DatabaseConfig = field(default_factory=DatabaseConfig)
    extra: Dict[str, Any] = field(default_factory=dict)
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'AppConfig':
        """Create config from dictionary."""
        db_data = data.pop('database', {})
        database = DatabaseConfig(**db_data) if db_data else DatabaseConfig()
        return cls(database=database, **data)
    
    @classmethod
    def from_json_file(cls, filepath: str) -> 'AppConfig':
        """Load config from JSON file."""
        with open(filepath, 'r') as f:
            data = json.load(f)
        return cls.from_dict(data)
    
    @classmethod
    def from_env(cls, prefix: str = "APP_") -> 'AppConfig':
        """Create config from environment variables."""
        return cls(
            app_name=os.environ.get(f"{prefix}NAME", "MyApp"),
            debug=os.environ.get(f"{prefix}DEBUG", "false").lower() == "true",
            log_level=os.environ.get(f"{prefix}LOG_LEVEL", "INFO"),
            database=DatabaseConfig(
                host=os.environ.get(f"{prefix}DB_HOST", "localhost"),
                port=int(os.environ.get(f"{prefix}DB_PORT", "5432")),
                database=os.environ.get(f"{prefix}DB_NAME", "mydb"),
                username=os.environ.get(f"{prefix}DB_USER", "user"),
                password=os.environ.get(f"{prefix}DB_PASS", ""),
            )
        )
    
    def to_dict(self) -> Dict[str, Any]:
        return asdict(self)

# Demo
config1 = AppConfig("MyWebApp", debug=True)
print("Config 1:")
print(config1)

config2 = AppConfig.from_dict({
    "app_name": "API Server",
    "debug": False,
    "database": {
        "host": "db.example.com",
        "port": 5432,
        "database": "production",
        "username": "admin",
        "password": "secret123"
    }
})
print(f"\nConfig 2:")
print(config2)
print(f"Connection: {config2.database.connection_string}")

### 8.2 HTTP Request Builder

In [ ]:
from typing import Dict, Optional, Any
from urllib.parse import urlencode

class HTTPRequest:
    """HTTP Request builder with fluent interface."""
    
    VALID_METHODS = ['GET', 'POST', 'PUT', 'DELETE', 'PATCH', 'HEAD', 'OPTIONS']
    
    def __init__(self, url: str, method: str = 'GET'):
        if method.upper() not in self.VALID_METHODS:
            raise ValueError(f"Invalid method: {method}")
        
        self.url = url
        self.method = method.upper()
        self._headers: Dict[str, str] = {}
        self._params: Dict[str, str] = {}
        self._body: Optional[Any] = None
        self._timeout: int = 30
    
    @classmethod
    def get(cls, url: str) -> 'HTTPRequest':
        return cls(url, 'GET')
    
    @classmethod
    def post(cls, url: str) -> 'HTTPRequest':
        return cls(url, 'POST')
    
    @classmethod
    def put(cls, url: str) -> 'HTTPRequest':
        return cls(url, 'PUT')
    
    @classmethod
    def delete(cls, url: str) -> 'HTTPRequest':
        return cls(url, 'DELETE')
    
    # Fluent interface methods (return self for chaining)
    def header(self, key: str, value: str) -> 'HTTPRequest':
        self._headers[key] = value
        return self
    
    def headers(self, headers: Dict[str, str]) -> 'HTTPRequest':
        self._headers.update(headers)
        return self
    
    def param(self, key: str, value: str) -> 'HTTPRequest':
        self._params[key] = value
        return self
    
    def params(self, params: Dict[str, str]) -> 'HTTPRequest':
        self._params.update(params)
        return self
    
    def body(self, data: Any) -> 'HTTPRequest':
        self._body = data
        return self
    
    def json(self, data: Dict) -> 'HTTPRequest':
        self._body = data
        self._headers['Content-Type'] = 'application/json'
        return self
    
    def timeout(self, seconds: int) -> 'HTTPRequest':
        self._timeout = seconds
        return self
    
    def auth_bearer(self, token: str) -> 'HTTPRequest':
        self._headers['Authorization'] = f'Bearer {token}'
        return self
    
    @property
    def full_url(self) -> str:
        if self._params:
            return f"{self.url}?{urlencode(self._params)}"
        return self.url
    
    def build(self) -> Dict[str, Any]:
        """Build the request dictionary."""
        return {
            'method': self.method,
            'url': self.full_url,
            'headers': self._headers,
            'body': self._body,
            'timeout': self._timeout
        }
    
    def __repr__(self):
        return f"HTTPRequest({self.method} {self.full_url})"

# Demo fluent interface
request = (
    HTTPRequest.post("https://api.example.com/users")
    .header("Accept", "application/json")
    .auth_bearer("my-secret-token")
    .json({"name": "John", "email": "john@example.com"})
    .timeout(60)
)

print(f"Request: {request}")
print(f"\nBuilt request:")
import json
print(json.dumps(request.build(), indent=2, default=str))

---

## 9. Best Practices và Common Mistakes

### 9.1 Best Practices

```python
# ✅ 1. Khởi tạo TẤT CẢ attributes trong __init__
def __init__(self, name):
    self.name = name
    self.items = []  # Không để None nếu sẽ dùng như list
    self._cache = None  # OK để dùng None cho optional

# ✅ 2. Validate trong __init__ hoặc property setters
def __init__(self, age):
    if age < 0:
        raise ValueError("Age cannot be negative")
    self.age = age

# ✅ 3. Dùng None cho mutable defaults
def __init__(self, items=None):
    self.items = items if items is not None else []

# ✅ 4. Dùng @classmethod cho alternative constructors
@classmethod
def from_string(cls, data_string):
    # Parse and return cls(...)
    pass

# ✅ 5. Document parameters với docstring
def __init__(self, name, age):
    """
    Initialize a Person.
    
    Args:
        name: Person's full name
        age: Age in years (must be >= 0)
    """
    pass
```

### 9.2 Common Mistakes

In [ ]:
# ❌ Mistake 1: Return value from __init__
class Bad1:
    def __init__(self, value):
        self.value = value
        # return self  # TypeError: __init__ should return None

# ❌ Mistake 2: Mutable default argument (đã demo ở trên)

# ❌ Mistake 3: Not calling parent __init__ in subclass
class Parent:
    def __init__(self, name):
        self.name = name

class BadChild(Parent):
    def __init__(self, name, age):
        # ❌ Quên gọi super().__init__()
        self.age = age

class GoodChild(Parent):
    def __init__(self, name, age):
        super().__init__(name)  # ✅ Gọi parent __init__
        self.age = age

# Test
try:
    bad = BadChild("Alice", 10)
    print(bad.name)  # AttributeError!
except AttributeError as e:
    print(f"BadChild error: {e}")

good = GoodChild("Bob", 12)
print(f"GoodChild: {good.name}, {good.age}")

---

## 10. Bài tập thực hành

### Bài 1: Temperature Class (Cơ bản)

Tạo class `Temperature` với:
- Constructor nhận Celsius
- Alternative constructors: from_fahrenheit(), from_kelvin()
- Properties: celsius, fahrenheit, kelvin

In [ ]:
class Temperature:
    """Temperature class với multiple constructors."""
    
    def __init__(self, celsius):
        # TODO: Validate và store celsius
        pass
    
    @classmethod
    def from_fahrenheit(cls, fahrenheit):
        # TODO: Convert F to C and create instance
        # Formula: C = (F - 32) * 5/9
        pass
    
    @classmethod
    def from_kelvin(cls, kelvin):
        # TODO: Convert K to C and create instance
        # Formula: C = K - 273.15
        pass
    
    @property
    def fahrenheit(self):
        # TODO: Return temperature in Fahrenheit
        pass
    
    @property
    def kelvin(self):
        # TODO: Return temperature in Kelvin
        pass

# Test (uncomment after implementing)
# t1 = Temperature(25)
# t2 = Temperature.from_fahrenheit(98.6)
# t3 = Temperature.from_kelvin(300)
# print(f"t1: {t1.celsius}°C = {t1.fahrenheit}°F = {t1.kelvin}K")

### Bài 2: User Registration (Trung bình)

Tạo class `User` với validation đầy đủ.

In [ ]:
import re
from datetime import datetime

class User:
    """User class với validation."""
    
    def __init__(self, username, email, password, age):
        """
        Create a new user.
        
        Validation rules:
        - username: 3-20 chars, alphanumeric + underscore
        - email: valid email format
        - password: min 8 chars, at least 1 uppercase, 1 lowercase, 1 digit
        - age: 13-120
        """
        # TODO: Implement with validation
        pass
    
    @classmethod
    def from_dict(cls, data):
        # TODO: Create from dictionary
        pass

# Test (uncomment after implementing)
# user = User("john_doe", "john@email.com", "SecurePass123", 25)
# print(user)

### Bài 3: Order System với Dataclass (Nâng cao)

Sử dụng dataclass để tạo Order system.

In [ ]:
from dataclasses import dataclass, field
from typing import List
from datetime import datetime
from enum import Enum

class OrderStatus(Enum):
    PENDING = "pending"
    CONFIRMED = "confirmed"
    SHIPPED = "shipped"
    DELIVERED = "delivered"
    CANCELLED = "cancelled"

@dataclass
class OrderItem:
    # TODO: Define fields: product_name, unit_price, quantity
    # TODO: Add property: subtotal
    pass

@dataclass
class Order:
    # TODO: Define fields: customer_name, items (list), status, created_at
    # TODO: Add __post_init__ for validation
    # TODO: Add properties: total, item_count
    # TODO: Add classmethod: from_dict
    pass

# Test (uncomment after implementing)
# order = Order(
#     customer_name="John Doe",
#     items=[
#         OrderItem("Laptop", 999.99, 1),
#         OrderItem("Mouse", 29.99, 2)
#     ]
# )
# print(f"Order total: ${order.total}")

---

## 11. Tổng kết

### Khái niệm chính

| Khái niệm | Mô tả |
|-----------|-------|
| **`__init__`** | Initializer - khởi tạo object đã được tạo |
| **`__new__`** | Constructor thực sự - tạo object mới |
| **Default params** | Giá trị mặc định cho optional arguments |
| **Validation** | Kiểm tra dữ liệu trong constructor |
| **@classmethod** | Tạo alternative constructors |
| **Dataclass** | Auto-generate boilerplate code |

### Khi nào dùng gì?

| Tình huống | Giải pháp |
|------------|------------|
| Object đơn giản với nhiều fields | `@dataclass` |
| Cần nhiều cách tạo object | `@classmethod` constructors |
| Cần validate input | Validate trong `__init__` hoặc property setters |
| Singleton pattern | Override `__new__` |
| Immutable object | `@dataclass(frozen=True)` hoặc subclass tuple/str |

### Tiếp theo: 03 - Inheritance (Kế thừa)

Trong bài tiếp theo, chúng ta sẽ học:
- Single và multiple inheritance
- Method overriding
- super() function
- Method Resolution Order (MRO)